# Reference:
  * [Simple Classification](https://github.com/MorvanZhou/Tensorflow-Tutorial/blob/master/tutorial-contents/302_simple_classification.py)
    1. 生成假資料
      * 二群。群一以(2, 2)為中心，lable=0；群二以(-2, -2)為中心，lable=1
      * 將群一與群二的資料合併, (x, y) = (坐標點, label)。
    2. 畫出原始資料的分佈。
    3. 建立神經網路
      * L1 層的輸入為 x(節點數為 2 = 坐標), 輸出的節點數為 10, activity_func=tf.nn.relu
      * output 層的輸入為 L1 的輸出(節點數為 10)，其輸出的節點數為 2，代表各為 lable=1或0 時的機率)
      * 定義 loss
      * 定義 accuracy
      * 執行
        * 注意 sess.run([train_op, accuracy, output], {tf_x: x, tf_y: y})，是指著並非所有定義的圖層都會被執行，而是只有這三個train_op, accuracy, output 圖層才會被執行而已
          * 其 return 值分別為以上三個 op 的結果。
          * 參考[session.run([fetch1, fetch2])](https://blog.csdn.net/u012436149/article/details/52908692)

In [ ]:
%matplotlib inline
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

tf.set_random_seed(1)
np.random.seed(1)

# fake data
n_data = np.ones((100, 2))
x0 = np.random.normal(2*n_data, 1)      # class0 x shape=(100, 2)
y0 = np.zeros(100)                      # class0 y shape=(100, )
x1 = np.random.normal(-2*n_data, 1)     # class1 x shape=(100, 2)
y1 = np.ones(100)                       # class1 y shape=(100, )
x = np.vstack((x0, x1))  # shape (200, 2) + some noise
y = np.hstack((y0, y1))  # shape (200, )

# plot data
plt.scatter(x[:, 0], x[:, 1], c=y, s=100, lw=0, cmap='RdYlGn')
plt.show()


tf_x = tf.placeholder(tf.float32, x.shape)     # input x
tf_y = tf.placeholder(tf.int32, y.shape)     # input y

# neural network layers
l1 = tf.layers.dense(tf_x, 10, tf.nn.relu)          # hidden layer
output = tf.layers.dense(l1, 2)                     # output layer
loss = tf.losses.sparse_softmax_cross_entropy(labels=tf_y, logits=output)           # compute cost
accuracy = tf.metrics.accuracy(          # return (acc, update_op), and create 2 local variables
    labels=tf.squeeze(tf_y), predictions=tf.argmax(output, axis=1),)[1]
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05)
train_op = optimizer.minimize(loss)

sess = tf.Session()                                                                 # control training and others
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init_op)     # initialize var in graph

# plt.ion()   # something about plotting
for step in range(100):
    # train and net output
    _, acc, pred = sess.run([train_op, accuracy, output], {tf_x: x, tf_y: y})
    if step % 20 == 0:
        # plot and show learning process
        plt.cla()
        plt.scatter(x[:, 0], x[:, 1], c=pred.argmax(1), s=100, lw=0, cmap='RdYlGn')
        plt.text(1.5, -4, 'Accuracy=%.2f' % acc, fontdict={'size': 20, 'color': 'red'})
        plt.pause(0.1)

# plt.ioff()
plt.show()